In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from models.baseline import CNN
from models.resnet import ResNetBase
from utils import read_all_images, read_labels
from sklearn import preprocessing
from sklearn.decomposition import PCA

sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)


In [3]:
X_train = read_all_images('../data/stl10_binary/train_X.bin').astype(np.float32)
X_test = read_all_images('../data/stl10_binary/test_X.bin').astype(np.float32)[:5000,...]
y_train = read_labels('../data/stl10_binary/train_y.bin')
y_test = read_labels('../data/stl10_binary/test_y.bin')[:5000,...]
input_shape = X_train.shape[1:]
print(input_shape)

Reading data...
Reading data...
(96, 96, 3)


## PCA transform

In [4]:
pca = PCA(n_components=128)
pca.fit(X_train.reshape(X_train.shape[0], -1))

X_train_pca = pca.transform(X_train.reshape(X_train.shape[0], -1))
X_test_pca = pca.transform(X_test.reshape(X_test.shape[0], -1))
print(X_train_pca.shape)
print(X_test_pca.shape)

(5000, 128)
(5000, 128)


## Baseline

In [5]:
clf = LogisticRegression(random_state=0, max_iter=1000, solver='lbfgs', multi_class='auto')
clf.fit(X_train_pca, y_train.squeeze())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
print("Train score:", clf.score(X_train_pca, y_train),
      "Test score:", clf.score(X_test_pca, y_test))

Train score: 0.428 Test score: 0.3558


## Self-Supervised pre-training

In [7]:
# encoder = CNN(input_shape, output_dim=128)
encoder = ResNetBase(input_shape, 128)

_ = encoder(np.random.rand(1,96,96,3).astype(np.float32), head='f')
_ = encoder(np.random.rand(1,96,96,3).astype(np.float32), head='g')
encoder.load_weights('../checkpoints/encoder.h5')

ValueError: axes don't match array

In [ ]:
X_train_repr, _ = encoder(X_train / 255, head=tf.constant('f'), training=tf.constant(False))
X_test_repr, _ = encoder(X_test / 255, head=tf.constant('f'), training=tf.constant(False))
print(X_train_repr.shape)
print(X_test_repr.shape)

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train_repr)
X_train_repr = scaler.transform(X_train_repr)
X_test_repr = scaler.transform(X_test_repr)

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=1000, solver='lbfgs', multi_class='auto')
clf.fit(X_train_repr, y_train.squeeze())

In [ ]:
print("Train score:", clf.score(X_train_repr, y_train.squeeze()))
print("Test score:", clf.score(X_test_repr, y_test.squeeze()))
# 0.48 0.4665
# 0.535625 0.4845

In [ ]:
X_train_repr[0]

In [ ]:
X_test_repr[0]